In [1]:
from Bio import Entrez
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
from pandas import DataFrame
# import plotly as py
import itertools
import csv 
import urllib
import codecs
import io
import sys
import encodings
# import chardet
import unicodedata
import time
import os
import math
from itertools import chain
import pyodbc
# import plotly.plotly as py
# import plotly.tools as plotly_tools
# from plotly.graph_objs import *
# import plotly.figure_factory as ff
from IPython.display import HTML
from datetime import date
import datetime
import sqlalchemy
from sqlalchemy import text
from sqlalchemy import bindparam
from datetime import datetime as dt
import types

# py.sign_in("jackp", "XXXX")

ModuleNotFoundError: No module named 'Bio'

# Create classes for fetching data

In [2]:
#searchParams formats search parameters for NCBI and generates and idList
class searchParams:
    def __init__(self,email, db):
        self.db = db
        self.email = email
        Entrez.email = email
        if db == "pubmed":
            self.__class__ = pubmedParams
        elif db == 'pmc':
            self.__class__ = pmcParams
            
            
class pubmedParams(searchParams):   
#in this case the term tags (e.g., TA) are the same for pubmed and PMC. but there are other tags that aren't the same.

    def get_term(self,journalTerm,keyTerm,useTags):
        """return the search term concatenated with type 😘"""
        journalType = '[TA]'
        keyType = "[ALL]"
        if useTags == 'No':
            if journalTerm != "" and keyTerm != "":
                term = journalTerm+" AND "+keyTerm
            elif journalTerm != "" and keyTerm == "":
                term = journalTerm
            elif keyTerm != "" and journalTerm == "":
                term = keyTerm
        elif useTags == 'Yes':
            if journalTerm != "" and keyTerm != "":
                term = journalTerm+journalType+" AND "+keyTerm+ keyType
            elif journalTerm != "" and keyTerm == "":
                term = journalTerm+journalType
            elif keyTerm != "" and journalTerm == "":
                term = keyTerm+ keyType
                
        return(term)
    
    def get_count(self, term):
        counthandle = Entrez.egquery(term = term)
        record = Entrez.read(counthandle)
        for row in record["eGQueryResult"]:
            if row['DbName'] == 'pubmed':
                rowcount = row['Count']
        counthandle.close()
        return rowcount
        
        
class pmcParams(searchParams):

    def get_term(self,journalTerm,keyTerm,useTags):
        """return the search term concatenated with type"""
        journalType = '[TA]'
        keyType = "[ALL]"
        if useTags == 'No':
            if journalTerm != "" and keyTerm != "":
                term = journalTerm+" AND "+keyTerm
            elif journalTerm != "" and keyTerm == "":
                term = journalTerm
            elif keyTerm != "" and journalTerm == "":
                term = keyTerm
        elif useTags == 'Yes':
            if journalTerm != "" and keyTerm != "":
                term = journalTerm+journalType+" AND "+keyTerm+ keyType
            elif journalTerm != "" and keyTerm == "":
                term = journalTerm+journalType
            elif keyTerm != "" and journalTerm == "":
                term = keyTerm+ keyType
        return(term)        
    
    
    def get_count(self, term):
        counthandle = Entrez.egquery(term = term)
        record = Entrez.read(counthandle)
        for row in record["eGQueryResult"]:
            if row['DbName'] == 'pmc':
                rowcount = row['Count']
        counthandle.close()
        return rowcount
        

In [3]:
#NCBISearch fetches the data from NCBI and ouutputs a list of "soups" for parsing
class NCBISearch:
    def __init__(self, email, retmode = 'xml'):
        self.email = email
        self.retmode = retmode
        Entrez.email = email

        #function for retrieving and storing ids
    def pub_search(self,term,db,recordCount,sort='relevance',chunksize = 500,sleeptime = 5):
        
        submitinterv = math.ceil(int(recordCount)/chunksize)
        
        #append to idlist
        idlist = []

        retstartc = 0
        for i in range(submitinterv):
            handle = Entrez.esearch(db=db, 
                                sort='relevance', 
                                retstart= retstartc,
                                retmax=chunksize,
                                retmode='xml', 
                                term=term)
            idresults = Entrez.read(handle)
            idlist += idresults['IdList']
            time.sleep(sleeptime)
            retstartc += chunksize
            handle.close()
                
        return idlist
    
    
    def pub_fetch(self, db, idList):
        soupList = []
        indices = [i for i, s in enumerate(idList)]
        for i in indices:
            ssHandle = Entrez.efetch(db=db,id = idList[i],sort='relevance', retstart= 0,retmax=300,
                    retmode='xml')
            ssHandle = io.TextIOWrapper(ssHandle.detach(), encoding='utf-8')
            ssResults = ssHandle.read()
            ssSoup = BeautifulSoup(ssResults, 'xml')
            soupList.append(ssSoup)
        return(soupList)

In [7]:
doc =  open('pmc_result.xml').read()


'<?xml version="1.0"?>\n<!DOCTYPE pmc-articleset PUBLIC "-//NLM//DTD ARTICLE SET 2.0//EN" "https://dtd.nlm.nih.gov/ncbi/pmc/articleset/nlm-articleset-2.0.dtd">\n<pmc-articleset>\n<article xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:xlink="http://www.w3.org/1999/xlink" article-type="research-article">\n  <?properties open_access?>\n  <front>\n    <journal-meta>\n      <journal-id journal-id-type="nlm-ta">BMC Public Health</journal-id>\n      <journal-id journal-id-type="iso-abbrev">BMC Public Health</journal-id>\n      <journal-title-group>\n        <journal-title>BMC Public Health</journal-title>\n      </journal-title-group>\n      <issn pub-type="epub">1471-2458</issn>\n      <publisher>\n        <publisher-name>BioMed Central</publisher-name>\n        <publisher-loc>London</publisher-loc>\n      </publisher>\n    </journal-meta>\n    <article-meta>\n      <article-id pub-id-type="pmid">28525991</article-id>\n      <article-id pub-id-type="pmc">5438549</article-id>\n      <a

In [27]:
doc.find('article')
import lxml.html
xml = lxml.html.fromstring(doc)
article = xml.xpath("//article")[0]
[elem for elem in dir(article) if "__" not in elem]
[elem.text_content() for elem in article.xpath('//article-id')]
thing = [elem for elem in article.xpath("//article-id/@pub-id-type")]
print(thing)

['pmid', 'pmc', 'publisher-id', 'doi']


In [4]:
#NCBIResults parses the output of what's pulled from the API and outputs a dataframe
class NCBIResults:
    def __init__(self, db):
        self.db = db
        if db == "pubmed":
            self.__class__ = pubmedFetch
        elif db == 'pmc':
            self.__class__ = pmcFetch

class pubmedFetch(NCBIResults):
      def parseArticle(self, soup):
            ArticleList = soup.find_all('PubmedArticle')
            articleList = []

            for article in ArticleList:
                ssList = []

                 ####### title ########
                title = article.find('ArticleTitle').text

                ####### pmid #########
                associatedId = article.find('PMID').text

                ####### journal ########
                journal = article.find('Title').text   

                ####### meshterms ##########

                #each meshterm will have its word and possibly some qualifiernames
                meshList = []

                try:
                    meshTermList = article.find_all('MeshHeading')
                    for term in meshTermList:
                        descriptorTerm = term.find('DescriptorName').text
                        qualifierList = []
                        try:
                            qualifierNameList = term.find_all('QualifierName')
                            for qual in qualifierNameList:
                                thisQual = qual.text
                                qualifierList.append(thisQual)
                        except: 
                            qualifierList = None

                        mtDict = {'term': descriptorTerm, 'qualList': qualifierList}
                        meshList.append(mtDict)
                except:
                    meshList = None

                ######## article date ###########

                if article.find('ArticleDate'):
                    year = article.find('DateCreated').find('Year').text
                    month = article.find('DateCreated').find('Month').text
                    day = article.find('DateCreated').find('Day').text
                    dateDict = {'month':month,'year':year,'day':day}
                    fulldate = '{0}/{1}/{2}'.format(month,day,year)
                if not article.find('ArticleDate'):
                    year = article.find('DateCreated').find('Year').text
                    month = article.find('DateCreated').find('Month').text
                    day = article.find('DateCreated').find('Day').text
                    dateDict = {'month':month,'year':year,'day':day}
                    fulldate = '{0}/{1}/{2}'.format(month,day,year)
                    



                ######### pubtype ##########
#                 pubTypeVal = ""
#                 try:
#                     pubTypes = article.find_all('PublicationType')
#                     for pub in pubTypes:
#                         try:
#                             pubTypeVal = pub.text
#                         except:
#                             pubTypeVal = None   
#                 except:
#                     pubTypeVal = None

                pubTypeVal = ""
                try:
                    pubTypes = article.find('PublicationType')
                    pubTypeVal = pubTypes.text
                except:
                    pubTypeVal = None
                    
                    
                ######### abstract ##########

                abstractList = []
                try:
                    abstractText = article.find('Abstract')
                    sections = abstractText.find_all('AbstractText')
                    if sections[0].get('Label'):
                        #broken up by nlm category
                        for sec in sections:
                            abstractDict = {}
                            abstractDict['NlmCategory'] = sec.get('NlmCategory')
                            abstractDict['Label'] = sec.get('Label')
                            abstractDict['text'] = sec.text
                            abstractList.append(abstractDict)
                    if not sections[0].get('Label'):
                        #comes as whole chunk
                        abstractDict = {}
                        abstractFull = abstractText.find('AbstractText').text
                        abstractDict['wholetext'] = abstractFull
                        abstractList.append(abstractDict)
                except:
                    #this means there is no abstract at all
                    abstractList = None

                ############ authors ###############

                #each article will have a list of authors. Each author is represented by a dictionary (key values: fname, lname, affliation)
                authList = []
                #some articles don't have an author
                try:          
                    authorList = article.find('AuthorList')
                    for author in authorList:
                        try:
                            affiliation = author.find('CollectiveName').text
                            lname = None
                            fname = None
                        except:
                            try:
                                lname = author.find('LastName').text
                            except:
                                lname = None
                            try:
                                fname = author.find('ForeName').text
                            except:
                                fname = None
                            try:
                                affiliation = author.find('Affiliation').text
                            except:
                                affiliation = None
                            authDict = {'lname':lname,'fname':fname, 'affl':affiliation}
                            authList.append(authDict)

                except:
                    authList = None

                ######### doi ##########
                doi = None

                try:
                    doiList = article.find_all('ArticleId')

                    for doiM in doiList:
                        if doiM.get('IdType') == 'doi':
                            doi = doiM.text
                except:

                    try:
                        doiList = article.find_all('ELocationID')
                        for doiM in doiList:
                            if doiM.get('IdType') == 'doi':
                                doi = doiM.text
                    except:
                        doi = None


                d = {
                        'title': title,
                        'associatedId' : associatedId,
                        'author': authList,    
                        'journal' : journal,
                        'pubtype': pubTypeVal,
                        'publishdate': fulldate,
                        'publishdatefull': dateDict,
                        'meshterms': meshList,
                        'abstract': abstractList,
                        'optionalId01' : doi,
                        'optionalId02': None
                 }

                articleList.append(d)
            return pd.DataFrame(articleList)
        
class pmcFetch(NCBIResults):
      def parseArticle(self, soup):

        ArticleList = soup.find_all('article')
        articleList = []
        for article in ArticleList:

            ####### title ########
            title = article.find('article-title').text.replace('\n', ' ')


            ####### journal ########
            journal = article.find('journal-title').text 

            ####### pmc id and doi id #########

            associatedId = None
            doi = None
            pmid = None
            litIdList=article.find_all('article-id')

            for lit in litIdList:
                idType = lit.get('pub-id-type')
                try:
                    if idType == 'pmc':
                        associatedId = lit.text
                except:
                    associatedId = None
                try:
                    if idType == 'doi':
                        doi = lit.text
                except:
                    doi = None
                try:
                    if idType == "pmid":
                        pmid = lit.text
                except:
                    pmid = None

            ######## article date ###########
            dateListInt = []
            dateDictList = []
            allDates = article.find_all('pub-date')
            for dates in allDates:
                try:    
                    month = dates.find('month').text
                except:
                    month = None
                try:
                    year = dates.find('year').text
                except:
                    year = None
                try:
                    day = dates.find('day').text
                except:
                    day = None
                dateDict = {'month':month, 'year':year, 'day':day}
                dateDictList.append(dateDict)
                fulldatesAll = '{0}/{1}/{2}'.format(month,day,year)
                dateListInt.append(fulldatesAll)


            dateValList = [num for num in dateListInt if 'None' not in num]
            if dateValList == []:
            #     dateValList = [num for num in dateListInt if 'None' in num]
                dateVal = None

            try:
                if isinstance(dateValList[0], str):
                    dateVal = dateValList[0]
                if isinstance(dateValList, str):
                    dateVal = dateValList
            except:
                dateVal = None


            dateDictValList  = [d for d in dateDictList if None not in d.values()]
            if not dateDictValList == []:
                ddList = dateDictValList
            if dateDictValList == []:
                ddList  = [d for d in dateDictList if None in d.values()]

            try:
                if isinstance(ddList[0],dict):
                    dateDictVal = ddList[0]
                if isinstance(ddList,dict):
                    dateDictVal = ddList
            except:
                dateDictVal = None


            ############ authors ###############     
            authList = []
            #to get the authors names
            try:
                contribList = article.find('contrib-group').find_all('name')
                for author in contribList:
                    try:
                        lname = author.find('surname').text
                    except:
                        lname = None
                    try:
                        fname = author.find('given-names').text
                    except:
                         fname = None     
                    authDict = {'lname':lname,'fname':fname, 'affl':None}
                    authList.append(authDict)
            except:
                authList = None


            ###### meshterms ##########

            #each meshterm will have its word and possibly some qualifiernames
            meshList = []

            try:
                meshTermList = article.find('kwd-group')
                wordList = meshTermList.find_all('kwd')
                for term in wordList:
                    descriptorTerm = term.text
                    mtDict = {'term': descriptorTerm, 'qualifiernames': None}
                    meshList.append(mtDict)
            except:
                meshTermList = None
            
            ######### publication type ##########
            pubTypeVal = ""
            try:
                pubTypes = article.find_all('subj-group')
                for pub in pubTypes:
                    try:
                        pubTypeVal = pub.find('subject').text
                    except:
                        pubTypeVal = None
            except:
                pubTypeVal = None
            
            ######### abstract ##########

            abstractList = []
            if article.find('abstract'):
                abstractText = article.find('abstract')
            if not article.find('abstract'):
                abstractText = article.find('body')

            try:
                sections = abstractText.find_all('sec')
                for sec in sections:
                    abstractDict = {}
                    abstractDict['NlmCategory'] = None
                    abstractDict['Label'] = sec.find('title').text.replace('\n', ' ')    
                    abstractDict['text'] = sec.find('p').text.replace('\n', ' ')    
                    abstractList.append(abstractDict)
            except:
                abstractList = []
                
            if abstractList == []:
                try:
                    pSections = abstractText.find_all('p')
                    pFull = ''
                    for p in pSections:
                        abstractDict = {}
                        pFull += p.text
                    abstractDict['wholetext'] = pFull
                    abstractList.append(abstractDict)
                except:
                    abstractList = []

            d = {
                        'title': title,
                        'associatedId' : associatedId,
                        'author': authList,    
                        'journal' : journal,
                        'pubtype': pubTypeVal,
                        'publishdate': dateVal,
                        'publishdatefull': dateDictVal,
                        'meshterms': meshList,
                        'abstract': abstractList,
                        'optionalId01' : doi,
                        'optionalId02': pmid
                     }

            articleList.append(d)
        return pd.DataFrame(articleList)

In [5]:
#ParseAll iterates through the list of soups, parses and compresses the outputs into one into dataframe
def ParseAll(db,soup):
    textReturns = NCBIResults(db)
    dfText = []
    for soupElt in soup:
        dfTextInst = textReturns.parseArticle(soupElt)
        dfText.append(dfTextInst)
        dfFull = pd.concat(dfText,ignore_index=True)
    return(dfFull)

# Set Search Parameters

In [215]:
#query db to find existing searches that were done and create unique ones
dbName = 'J:/ssUseCases/Database6.accdb'
connStr = 'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=%s;' % dbName
cnxn = pyodbc.connect(connStr)
cur = cnxn.cursor()
query = "Select distinct PullQuery,PullSource from DataPull_ID"
pullDf = pd.read_sql_query(query,cnxn)
pd.set_option('display.max_colwidth',100)
pullDf.sort_values(by = 'PullSource')

PullQuery  \
0                                      (syndromic+surveillance AND emergency+department) AND west+nile   
13                                              syndromic+surveillance AND emergency+department+visits   
12                                                 syndromic+surveillance AND emergency+department[TI]   
10                                                     syndromic+surveillance AND emergency+department   
15                                                                          syndromic+surveillance[TI]   
6                                              syndromic+surveillance AND (emergency+department OR ED)   
8                                  syndromic+surveillance AND (emergency+department OR emergency+room)   
4                                    Online J Public Health Inform[TA] AND syndromic+surveillance[ALL]   
2   Online J Public Health Inform[TA] AND syndromic+surveillance AND (emergency+department OR ED)[ALL]   
1                                         Online J Public Health Inform AND syndromic+surveillance[TI]   
7                                              syndromic+surveillance AND (emergency+department OR ED)   
9                                  syndromic+surveillance AND (emergency+department OR emergency+room)   
3   Online J Public Health Inform[TA] AND syndromic+surveillance AND (emergency+department OR ED)[ALL]   
11                                                     syndromic+surveillance AND emergency+department   
14                                              syndromic+surveillance AND emergency+department+visits   
5                                    Online J Public Health Inform[TA] AND syndromic+surveillance[ALL]   
16                                                                          syndromic+surveillance[TI]   

   PullSource  
0         PMC  
13        PMC  
12        PMC  
10        PMC  
15        PMC  
6         PMC  
8         PMC  
4         PMC  
2         PMC  
1         PMC  
7      PUBMED  
9      PUBMED  
3      PUBMED  
11     PUBMED  
14     PUBMED  
5      PUBMED  
16     PUBMED

In [216]:
email = r's.rand525@gmail.com'
# journalTerm = 'Online J Public Health Inform'
# keyTerm = r'syndromic+surveillance AND emergency+department+visits' H
# keyTerm = r'(syndromic+surveillance AND emergency+department) AND west+nile' 
# keyTerm = r'syndromic+surveillance AND emergency+department'  H
# keyTerm = r'syndromic+surveillance AND (emergency+department OR ED)'  
# keyTerm = r'syndromic+surveillance AND (emergency+department OR emergency+room)' H
# keyTerm = 'syndromic+surveillance'
#to make a title search
journalTerm = ''
keyTerm = 'syndromic+surveillance[TIAB]'
# keyTerm = r'syndromic+surveillance AND emergency+department[TI]'
db = 'pubmed'
useTags = 'No'

# Fetch Data

In [217]:
#Generate concatenated search term and idList
pmParamObj = searchParams(email, db)
pmSearchTerm = pmParamObj.get_term(journalTerm,keyTerm,useTags)
pmCount = pmParamObj.get_count(term = pmSearchTerm)
pmSearchObj = NCBISearch(email)
idList = pmSearchObj.pub_search(term = pmSearchTerm, recordCount=pmCount,chunksize=50000, db=db)

In [220]:
#make soup for parsing
soup = pmSearchObj.pub_fetch(db = db, idList = idList)

In [221]:
#Parse soup into dataframe
dfFull = ParseAll(db,soup)

In [222]:
dfFull.head()

abstract  \
0                                                                                                 None   
1  [{'wholetext': 'During winter 2014-15, England experienced severe strains on acute health servic...   
2  [{'wholetext': 'The eradication of viral haemorrhagic septicaemia virus (VHSV Id) from Finnish b...   
3  [{'NlmCategory': 'OBJECTIVE', 'Label': 'OBJECTIVE', 'text': 'To investigate the public health do...   
4  [{'wholetext': 'Objective The objective was to forecast and validate prediction estimates of inf...   

  associatedId  \
0     29057228   
1     29048277   
2     29044041   
3     29026457   
4     29026453   

                                                                                                author  \
0  [{'lname': None, 'fname': None, 'affl': None}, {'lname': 'Barrett', 'fname': 'Damien', 'affl': '...   
1  [{'lname': None, 'fname': None, 'affl': None}, {'lname': 'Smith', 'fname': 'Sue', 'affl': None},...   
2  [{'lname': None, 'fname': None, 'affl': None}, {'lname': 'Vennerström', 'fname': 'Pia', 'affl': ...   
3  [{'lname': None, 'fname': None, 'affl': None}, {'lname': 'Bhattarai', 'fname': 'Arjun Kumar', 'a...   
4  [{'lname': None, 'fname': None, 'affl': None}, {'lname': 'Paul', 'fname': 'Susannah', 'affl': 'R...   

                                       journal meshterms  \
0              Frontiers in veterinary science        []   
1                 Emerging infectious diseases        []   
2                Diseases of aquatic organisms        []   
3  Online journal of public health informatics        []   
4  Online journal of public health informatics        []   

               optionalId01 optionalId02 publishdate  \
0  10.3389/fvets.2017.00150         None  10/23/2017   
1    10.3201/eid2311.161632         None  10/19/2017   
2          10.3354/dao03161         None  10/18/2017   
3   10.5210/ojphi.v9i2.7985         None  10/13/2017   
4   10.5210/ojphi.v9i2.8004         None  10/13/2017   

                                publishdatefull          pubtype  \
0  {'month': '10', 'year': '2017', 'day': '23'}  Journal Article   
1  {'month': '10', 'year': '2017', 'day': '19'}  Journal Article   
2  {'month': '10', 'year': '2017', 'day': '18'}  Journal Article   
3  {'month': '10', 'year': '2017', 'day': '13'}  Journal Article   
4  {'month': '10', 'year': '2017', 'day': '13'}  Journal Article   

                                                                                                 title  
0                                The Potential for Big Data in Animal Disease Surveillance in Ireland.  
1  Retrospective Observational Study of Atypical Winter Respiratory Illness Season Using Real-Time ...  
2  Viral haemorrhagic septicaemia virus (VHSV Id) infections are detected more consistently using s...  
3  Applications of information and communications technologies to public health: A scoping review u...  
4  Modeling and Forecasting Influenza-like Illness (ILI) in Houston, Texas Using Three Surveillance...

# Create classes for pushing data to access database

In [49]:
#dbName = access database name
class NCBIPush:
    def __init__(self, dbName):
        self.dbName = dbName
        
        #function for connecting to database
    def dbConnect(self):
        #Connect to access database
#        dbString = "J:/ssUseCases/Database3.accdb"
        connStr = 'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=%s;' % self.dbName
        cnxn = pyodbc.connect(connStr)
        cur = cnxn.cursor()
        return(cnxn, cur)
        
    #Determining Pull Id
    def pullId (self,cnxn):
        query = "select max(PullID) from DataPull_ID"
        pullIdDf = pd.read_sql_query(query,cnxn)
        pullIdDf.columns = ['maxPullId']
        idVal = pullIdDf['maxPullId'].isnull()
        if idVal.bool():
            pullVal = 1
        else:
            pullIdSeries = pullIdDf['maxPullId'] + 1
            pullVal = pullIdSeries.iloc[0]
        return(pullVal)
    
    #DataPull_ID table
    def DataPull_ID(self,pullVal,cnxn,cur):
        try:
            pullId = pullVal.item()
        except:
            pullId = pullVal
#         pullDate = date.today().strftime('%m/%d/%Y')
        pullDate = datetime.datetime.now().strftime('%m/%d/%Y %I:%M:%S %p')
        pullName = 'ss use cases'
        pullQuery = pmSearchTerm
        pullType = 'keyword' #keyword/ author
        pullSource = db.upper()
        pullBy = 'Sophie'

        query = """ insert into DataPull_ID (PullID,PullDate,PullName,PullQuery,PullType,PullSource,PullBy) 
                    values (?,?,?,?,?,?,?) """

        args = (pullId,pullDate,pullName,pullQuery,pullType,pullSource,pullBy)

        cur.execute(query, args)
        #must commit in order to see it on sql server, if not sql server database won't load correctly
        cnxn.commit()
    
    #DataPull_Detail table
    def DataPull_Detail(self,cnxn,cur,dfFull):
        accessPK = cur.execute("select @@IDENTITY").fetchall()[0][0].__str__()
        pullId = cur.execute("select distinct PullID from DataPull_ID where ID = ?", (accessPK)).fetchall()[0][0].__str__()

        #         #get all AssociatedIDs already in the database from same source
        aidList = cur.execute("""select distinct b.AssociatedID from DataPull_ID as a inner join DataPull_Detail as b on a.PullID = b.PullID where a.PullSource = ?""",db).fetchall()
        existing_ids = {associatedid[0] for associatedid in aidList}


        query = """ insert into DataPull_Detail ([PullID],[AssociatedID],[ValueStore],[PubType],[Note]) 
                            values (?,?,?,?,?) """

        note = None

        values_list = []

        #         #check how many associatedIDs there are
        for index,row in dfFull.iterrows():

            associatedidInt = int(row['associatedId'])
            associatedid = str(associatedidInt)
            pubtype = row['pubtype']
            #check if associatedid is already in the database
            if associatedid in existing_ids:
                valuestore = 'duplicate'
                #drop the row that already exists
                dfFull.drop(index, inplace=True)
            else:
                valuestore = 'store'

            values_list.append((pullId,associatedid,valuestore,pubtype,note))

        if values_list!=[]:
            cur.executemany(query, values_list)
            cnxn.commit()

    def DataPull_Title(self,cnxn,cur,dfFull):
        query = """ insert into DataPull_Title (AssociatedID, Title, Journal, PublicationDate, pubDay,pubMonth,pubYear, OptionalID01,OptionalID02) values (?,?,?,?,?,?,?,?,?) """

        values_list = []

        for index,row in dfFull.iterrows():
            associatedid = int(row['associatedId'])
            title = row['title']
            journal = row['journal']
            pubdate = row['publishdate']
            doi = row['optionalId01']
            pmid = row['optionalId02']    
            if row['publishdatefull'] is not None:
                dictDate = row['publishdatefull']
                pubDay = dictDate['day']
                pubMonth = dictDate['month']
                pubYear = dictDate['year']

            values_list.append((associatedid, title, journal ,pubdate,pubDay,pubMonth,pubYear, doi,pmid))


        if values_list!=[]:
            cur.executemany(query, values_list)
            cnxn.commit()

            
            
    def DataPull_Keyword(self,cnxn,cur,dfFull):
        query = """ insert into DataPull_Keyword (AssociatedID, KeywordValue, Category1, Category2,
                    Category3, Category4, Category5) values (?,?,?,?,?,?,?)"""
        values_list = []

        for index,row in dfFull.iterrows():
            associatedid = int(row['associatedId'])

            if row['meshterms'] is not None:

                #each meshterm comes in as a dictionary (keys: qualifiernames & term)
                #qualifiernames value = a list of qualifernames
                #term value = the actual mesh term
                for word in row['meshterms']:
                    keywordvalue = word['term']

                    try:
                        num_of_qualifiers = len(word['qualifiernames'])
                    except:
                        num_of_qualifiers = 0

                    #some have more than 5 qualifier names, if that is the case, then make num_of_Nones to 0 
                    #needed to fill up ? marks with NULLS

                    if num_of_qualifiers > 5:
                        num_of_Nones = 0
                        word['qualifiernames'] = word['qualifiernames'][:5]
                    else:
                        num_of_Nones = 5 - num_of_qualifiers

                    #if there are qualifier names, the list should not be 0

                    if num_of_qualifiers != 0:
                        values_list.append(([associatedid,keywordvalue] + word['qualifiernames'] + [None]*num_of_Nones)) 
                    else:
                        values_list.append((associatedid, keywordvalue, None, None, None, None, None))
 
        if values_list!=[]:
            cur.executemany(query, values_list)
            cnxn.commit()
        
    def DataPull_Authors(self,cnxn,cur,dfFull):
        #DataPull_Authors table
        query = """ insert into DataPull_Author (AssociatedID, ForeName, LastName, Affiliation) values (?,?,?,?) """

        values_list = []

        for index,row in dfFull.iterrows():
            associatedid = int(row['associatedId'])
            auth_count = 0

            if row['author'] is not None:
                for auth in row['author']:
                    auth_count += 1
                    if auth_count < 4:
                        values_list.append((associatedid, auth['fname'], auth['lname'], auth['affl']))
                    else:
                        break

        if values_list!=[]:
            cur.executemany(query, values_list)
            cnxn.commit()
        
        
    def DataPull_Text(self,cnxn,cur,dfFull):        
        
        query = """ insert into DataPull_Text (associatedid, nlmcategory, label, abstracttext) values (?,?,?,?)"""

        values_list = []

        for index,row in dfFull.iterrows():
            associatedid = int(row['associatedId'])

            if row['abstract'] is not None:
                for part in row['abstract']:

                    #check if the abstract is just one chunk
                    try:
                        values_list.append((associatedid, None, None, part['wholetext']))
                    except:
                        values_list.append((associatedid, part['NlmCategory'], part['Label'], part['text']))

        if values_list!=[]:
            cur.executemany(query, values_list)
            cnxn.commit()

In [223]:
#dbString - should be the access database location
dbString = "J:/ssUseCases/Database6.accdb"
inst= NCBIPush(dbString)
cn = inst.dbConnect()
cnxn = cn[0]
cur = cn[1]

In [224]:
val = inst.pullId(cnxn)

In [225]:
val

20

In [151]:
#Function to run all push methods of NCBIPush instance.
def pushData(objInst,cnxn,cur,df):
    #Generate new primary key 
    val = objInst.pullId(cnxn)
    
    #Push data into access database
    objInst.DataPull_ID(val,cnxn,cur)
    objInst.DataPull_Detail(cnxn,cur,df)
    objInst.DataPull_Title(cnxn,cur,df)
    objInst.DataPull_Keyword(cnxn,cur,df)
    objInst.DataPull_Authors(cnxn,cur,df)
    objInst.DataPull_Text(cnxn,cur,df)
    return(val)

In [226]:
a=pushData(inst,cnxn,cur,dfFull)

In [89]:
a

8

In [50]:
val

4